# Imports

In [97]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Recall
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

import pickle as pkl

from sklearn.metrics import recall_score, accuracy_score

# Load data

In [2]:
df_train = pd.read_csv('data/ready/train_rf.csv')
df_train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,isna_MonthlyIncome,isna_NumberOfDependents
0,0,0.479144,28,0,0.749607,2543.0,9,0,2,2,0.0,True,True
1,0,0.045562,59,2,0.921104,4600.0,23,0,1,0,0.0,True,True
2,0,0.046039,43,0,0.002092,5734.0,2,0,0,0,1.0,True,True
3,0,0.000043,66,2,1196.000000,5400.0,9,0,1,0,0.0,True,True
4,0,0.556032,49,0,0.496295,8500.0,17,0,1,0,0.0,True,True


In [3]:
df_val = pd.read_csv('data/ready/val_rf.csv')
df_val.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,isna_MonthlyIncome,isna_NumberOfDependents
0,0,1.922615,34,0,722.000000,5400.0,10,0,0,0,0.0,True,True
1,0,0.886104,75,1,0.489588,4321.0,15,0,2,0,1.0,True,True
2,0,0.223532,62,0,0.229667,8446.0,13,0,1,0,1.0,True,True
3,0,0.000000,71,0,2521.000000,5400.0,10,0,1,1,1.0,True,True
4,0,0.396280,51,0,0.592902,7550.0,12,0,2,0,4.0,True,True


# Scale

In [5]:
scaler = StandardScaler()

cols_x = list(set(df_train) - set(['SeriousDlqin2yrs']))

x_train = scaler.fit_transform(df_train[cols_x])

In [8]:
x_val = scaler.transform(df_val[cols_x])

# Model

In [43]:
class_weights = (df_train['SeriousDlqin2yrs'].value_counts() / len(df_train)).to_dict()

weight_0 = class_weights[0]
class_weights[0] = class_weights[1]
class_weights[1] = weight_0

class_weights

{0: 0.06674285714285715, 1: 0.9332571428571429}

## Find good lr

In [44]:
model = Sequential()

model.add(
        Dense(100
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
              metrics=[recall, 'accuracy'])

In [45]:
model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 3s 655us/step - loss: 0.0663 - recall_10: 0.6677 - accuracy: 0.8049 - val_loss: 0.5192 - val_recall_10: 0.7006 - val_accuracy: 0.8091
Epoch 2/10
4375/4375 [==============================] - 3s 633us/step - loss: 0.0649 - recall_10: 0.6671 - accuracy: 0.8264 - val_loss: 0.5132 - val_recall_10: 0.6456 - val_accuracy: 0.8396
Epoch 3/10
4375/4375 [==============================] - 3s 627us/step - loss: 0.0643 - recall_10: 0.6708 - accuracy: 0.8237 - val_loss: 0.5776 - val_recall_10: 0.7475 - val_accuracy: 0.7381
Epoch 4/10
4375/4375 [==============================] - 3s 627us/step - loss: 0.0636 - recall_10: 0.6628 - accuracy: 0.8289 - val_loss: 0.5616 - val_recall_10: 0.6538 - val_accuracy: 0.8467
Epoch 5/10
4375/4375 [==============================] - 3s 628us/step - loss: 0.0642 - recall_10: 0.7149 - accuracy: 0.7921 - val_loss: 0.4722 - val_recall_10: 0.6375 - val_accuracy: 0.8507
Epoch 6/10
4375/4375 [============================

In [46]:
model = Sequential()

model.add(
        Dense(100
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 3s 653us/step - loss: 0.0885 - recall_11: 0.5111 - accuracy: 0.4983 - val_loss: 0.7174 - val_recall_11: 1.0000 - val_accuracy: 0.0703
Epoch 2/10
4375/4375 [==============================] - 3s 626us/step - loss: 0.0882 - recall_11: 0.5034 - accuracy: 0.4928 - val_loss: 0.5851 - val_recall_11: 0.0000e+00 - val_accuracy: 0.9299
Epoch 3/10
4375/4375 [==============================] - 3s 627us/step - loss: 0.0868 - recall_11: 0.5238 - accuracy: 0.4827 - val_loss: 0.7169 - val_recall_11: 1.0000 - val_accuracy: 0.0701
Epoch 4/10
4375/4375 [==============================] - 3s 625us/step - loss: 0.0869 - recall_11: 0.4996 - accuracy: 0.4914 - val_loss: 0.6636 - val_recall_11: 0.0000e+00 - val_accuracy: 0.9299
Epoch 5/10
4375/4375 [==============================] - 3s 625us/step - loss: 0.0868 - recall_11: 0.4887 - accuracy: 0.5176 - val_loss: 0.6578 - val_recall_11: 0.0000e+00 - val_accuracy: 0.9299
Epoch 6/10
4375/4375 [================

In [47]:
model = Sequential()

model.add(
        Dense(100
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 3s 658us/step - loss: 0.0659 - recall_12: 0.6849 - accuracy: 0.7916 - val_loss: 0.4824 - val_recall_12: 0.6884 - val_accuracy: 0.8229
Epoch 2/10
4375/4375 [==============================] - 3s 633us/step - loss: 0.0632 - recall_12: 0.6922 - accuracy: 0.8137 - val_loss: 0.4472 - val_recall_12: 0.6538 - val_accuracy: 0.8430
Epoch 3/10
4375/4375 [==============================] - 3s 634us/step - loss: 0.0628 - recall_12: 0.6956 - accuracy: 0.8142 - val_loss: 0.5173 - val_recall_12: 0.7169 - val_accuracy: 0.7953
Epoch 4/10
4375/4375 [==============================] - 3s 633us/step - loss: 0.0624 - recall_12: 0.7052 - accuracy: 0.8116 - val_loss: 0.5519 - val_recall_12: 0.7230 - val_accuracy: 0.7939
Epoch 5/10
4375/4375 [==============================] - 3s 632us/step - loss: 0.0623 - recall_12: 0.7083 - accuracy: 0.8102 - val_loss: 0.4865 - val_recall_12: 0.7006 - val_accuracy: 0.8003
Epoch 6/10
4375/4375 [============================

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1),
              metrics=[recall])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

1e-2 it is

## Bigger network

In [48]:
model = Sequential()

model.add(
        Dense(100
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 4s 778us/step - loss: 0.0657 - recall_13: 0.6848 - accuracy: 0.7943 - val_loss: 0.5413 - val_recall_13: 0.7189 - val_accuracy: 0.7863
Epoch 2/10
4375/4375 [==============================] - 3s 742us/step - loss: 0.0633 - recall_13: 0.6934 - accuracy: 0.8116 - val_loss: 0.4554 - val_recall_13: 0.6741 - val_accuracy: 0.8379
Epoch 3/10
4375/4375 [==============================] - 3s 741us/step - loss: 0.0628 - recall_13: 0.6984 - accuracy: 0.8118 - val_loss: 0.5581 - val_recall_13: 0.7434 - val_accuracy: 0.7667
Epoch 4/10
4375/4375 [==============================] - 3s 741us/step - loss: 0.0625 - recall_13: 0.7097 - accuracy: 0.8033 - val_loss: 0.4894 - val_recall_13: 0.7312 - val_accuracy: 0.7796
Epoch 5/10
4375/4375 [==============================] - 3s 745us/step - loss: 0.0623 - recall_13: 0.7100 - accuracy: 0.8074 - val_loss: 0.4978 - val_recall_13: 0.7271 - val_accuracy: 0.7781
Epoch 6/10
4375/4375 [============================

In [50]:
model = Sequential()

model.add(
        Dense(100
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 4s 881us/step - loss: 0.0660 - recall_15: 0.6864 - accuracy: 0.7938 - val_loss: 0.4861 - val_recall_15: 0.6721 - val_accuracy: 0.8301
Epoch 2/10
4375/4375 [==============================] - 4s 854us/step - loss: 0.0633 - recall_15: 0.6963 - accuracy: 0.8070 - val_loss: 0.5659 - val_recall_15: 0.7658 - val_accuracy: 0.7413
Epoch 3/10
4375/4375 [==============================] - 4s 853us/step - loss: 0.0628 - recall_15: 0.7074 - accuracy: 0.8029 - val_loss: 0.5071 - val_recall_15: 0.6925 - val_accuracy: 0.8183
Epoch 4/10
4375/4375 [==============================] - 4s 852us/step - loss: 0.0626 - recall_15: 0.7083 - accuracy: 0.8054 - val_loss: 0.5419 - val_recall_15: 0.7230 - val_accuracy: 0.7777
Epoch 5/10
4375/4375 [==============================] - 4s 853us/step - loss: 0.0625 - recall_15: 0.7048 - accuracy: 0.8098 - val_loss: 0.5016 - val_recall_15: 0.7128 - val_accuracy: 0.8039
Epoch 6/10
4375/4375 [============================

## Add batchNorm

In [53]:
model = Sequential()

model.add(
        Dense(200
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 8s 2ms/step - loss: 0.0685 - recall_17: 0.6931 - accuracy: 0.7598 - val_loss: 0.4752 - val_recall_17: 0.6090 - val_accuracy: 0.8646
Epoch 2/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0651 - recall_17: 0.7009 - accuracy: 0.7894 - val_loss: 0.4998 - val_recall_17: 0.6986 - val_accuracy: 0.8019
Epoch 3/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0640 - recall_17: 0.7013 - accuracy: 0.8018 - val_loss: 0.4621 - val_recall_17: 0.6415 - val_accuracy: 0.8619
Epoch 4/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0636 - recall_17: 0.7045 - accuracy: 0.7998 - val_loss: 0.5626 - val_recall_17: 0.7821 - val_accuracy: 0.7457
Epoch 5/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0628 - recall_17: 0.7108 - accuracy: 0.7987 - val_loss: 0.4905 - val_recall_17: 0.7047 - val_accuracy: 0.8149
Epoch 6/10
4375/4375 [==============================] - 7s 2

## Adjust class weights

In [57]:
class_weights = {0: 0.05, 1: 0.95}

In [58]:
model = Sequential()

model.add(
        Dense(200
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=10
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0598 - recall_19: 0.7566 - accuracy: 0.6830 - val_loss: 0.5116 - val_recall_19: 0.6680 - val_accuracy: 0.8231
Epoch 2/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0565 - recall_19: 0.7627 - accuracy: 0.7247 - val_loss: 0.5635 - val_recall_19: 0.6802 - val_accuracy: 0.8259
Epoch 3/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0558 - recall_19: 0.7673 - accuracy: 0.7275 - val_loss: 0.5685 - val_recall_19: 0.7373 - val_accuracy: 0.7687
Epoch 4/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0552 - recall_19: 0.7699 - accuracy: 0.7326 - val_loss: 0.6053 - val_recall_19: 0.7862 - val_accuracy: 0.7007
Epoch 5/10
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0550 - recall_19: 0.7753 - accuracy: 0.7340 - val_loss: 0.5625 - val_recall_19: 0.7210 - val_accuracy: 0.7866
Epoch 6/10
4375/4375 [==============================] - 7s 2

## More epochs

In [122]:
model = Sequential()

model.add(
        Dense(200
          , input_shape=(len(cols_x),) 
          , activation='relu'
            )
         )

model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(200, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

recall = Recall()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[recall, 'accuracy'])

model.fit(x_train,df_train['SeriousDlqin2yrs'],epochs=15
          , class_weight=class_weights
          , validation_data=(x_val, df_val['SeriousDlqin2yrs']))

Epoch 1/15
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0598 - recall_25: 0.7590 - accuracy: 0.6781 - val_loss: 0.5112 - val_recall_25: 0.6741 - val_accuracy: 0.8260
Epoch 2/15
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0567 - recall_25: 0.7602 - accuracy: 0.7233 - val_loss: 0.6585 - val_recall_25: 0.7862 - val_accuracy: 0.6840
Epoch 3/15
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0560 - recall_25: 0.7637 - accuracy: 0.7340 - val_loss: 0.5545 - val_recall_25: 0.7047 - val_accuracy: 0.8109
Epoch 4/15
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0553 - recall_25: 0.7728 - accuracy: 0.7307 - val_loss: 0.5567 - val_recall_25: 0.7189 - val_accuracy: 0.7951
Epoch 5/15
4375/4375 [==============================] - 7s 2ms/step - loss: 0.0550 - recall_25: 0.7687 - accuracy: 0.7406 - val_loss: 0.6458 - val_recall_25: 0.7984 - val_accuracy: 0.7001
Epoch 6/15
4375/4375 [==============================] - 7s 2

In [123]:
model.save('models/keras_model')

INFO:tensorflow:Assets written to: models/keras_model/assets


# Test

In [124]:
df_test = pd.read_csv('data/raw/test.csv')
df_test.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.122890,75,0,0.064911,5884.0,3,0,0,0,0.0
1,0,0.917541,27,0,0.059415,5570.0,5,0,0,0,0.0
2,0,0.000000,66,0,0.000000,NaN,4,0,0,0,0.0
3,1,1.004183,33,0,1.151283,3000.0,12,0,3,1,0.0
4,1,0.745243,57,0,0.268780,3500.0,9,0,1,1,0.0


In [125]:
with open('models/missings.pkl', 'rb') as handle:
    miss_dict = pkl.load(handle)

In [126]:
miss_dict

{'MonthlyIncome': 5400.0, 'NumberOfDependents': 0.0}

In [127]:
df_test.isna().sum()

SeriousDlqin2yrs                          0
RevolvingUtilizationOfUnsecuredLines      0
age                                       0
NumberOfTime30-59DaysPastDueNotWorse      0
DebtRatio                                 0
MonthlyIncome                           624
NumberOfOpenCreditLinesAndLoans           0
NumberOfTimes90DaysLate                   0
NumberRealEstateLoansOrLines              0
NumberOfTime60-89DaysPastDueNotWorse      0
NumberOfDependents                       77
dtype: int64

In [128]:
for k in miss_dict.keys():
    df_test[f'isna_{k}'] = 0
    df_test.loc[df_test[k].isna(), f'isna_{k}'] = True
    df_test.loc[df_test[k].isna(), k] = miss_dict[k]
    
df_test.isna().sum()

SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
isna_MonthlyIncome                      0
isna_NumberOfDependents                 0
dtype: int64

In [129]:
x_test = scaler.transform(df_test[cols_x])

In [167]:
print('Train recall:',
    model.evaluate(x_train, df_train['SeriousDlqin2yrs'])[1]
     )

4375/4375 [==============================] - 4s 871us/step - loss: 0.5128 - recall_25: 0.7939 - accuracy: 0.7737
Train recall: 0.793878436088562


In [168]:
print('Validation recall:',
    model.evaluate(x_val, df_val['SeriousDlqin2yrs'])[1]
     )

219/219 [==============================] - 0s 776us/step - loss: 0.5103 - recall_25: 0.7943 - accuracy: 0.7760
Validation recall: 0.7942973375320435


In [169]:
print('Test recall:',
    model.evaluate(x_test, df_test['SeriousDlqin2yrs'])[1]
     )

94/94 [==============================] - 0s 807us/step - loss: 0.5262 - recall_25: 0.7382 - accuracy: 0.7610
Test recall: 0.7382199168205261


# Save scaler

In [170]:
with open('models/nn_scaler.pkl', 'wb') as handle:
    pkl.dump(scaler, handle)